### 1. Import thư viện cần thiết

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

### 2. Khởi tạo Spark 

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IE212_Lab2_Bai2") \
    .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/11/25 21:56:32 WARN Utils: Your hostname, ELUCIDATOR resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/25 21:56:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 21:56:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/25 21:56:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### 3. Load dữ liệu

In [3]:
movies_path = "data/movies.txt"
ratings1_path = "data/ratings_1.txt"
ratings2_path = "data/ratings_2.txt"

In [4]:
movies_rdd = sc.textFile(movies_path)

In [5]:
ratings_rdd = sc.textFile(f"{ratings1_path},{ratings2_path}")

### 4. Giải quyết bài toán

##### 4.1. Các hàm hỗ trợ

In [6]:
# 2. Xử lý Ratings: Tính tổng điểm và số lượt đánh giá cho MỖI PHIM trước
# Lý do: Gom nhóm theo phim trước sẽ giảm kích thước dữ liệu trước khi Join, giúp chạy nhanh hơn.
def parse_rating_entry(line):
    # Schema: UserID, MovieID, Rating, Timestamp
    parts = line.split(",")
    return (parts[1], (float(parts[2]), 1))

# Kết quả: (MovieID, (SumRating, Count))
movie_rating_totals = ratings_rdd.map(parse_rating_entry).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [7]:
# 3. Xử lý Movies: Tách (Explode) thể loại bằng flatMap
# Input: "1001,The Godfather (1972),Crime|Drama"
# Output mong muốn: [('1001', 'Crime'), ('1001', 'Drama')]
def extract_genres(line):
    # Schema: MovieID, Title, Genres
    parts = line.split(",")
    movie_id = parts[0]
    raw_genres = parts[2]
    
    # Tách chuỗi genre bằng dấu "|"
    genre_list = raw_genres.split("|")
    
    # Tạo list các cặp (MovieID, Genre)
    output_pairs = []
    for genre in genre_list:
        output_pairs.append((movie_id, genre))
    
    return output_pairs

##### 4.2. Xử lý tính toán

In [8]:
# Sử dụng flatMap để biến 1 dòng phim thành N dòng thể loại
movie_genres_rdd = movies_rdd.flatMap(extract_genres)

In [9]:
# 4. Join dữ liệu: Ghép Thể loại với Thống kê Rating của phim
# movie_genres_rdd:    (MovieID, Genre)
# movie_rating_totals: (MovieID, (SumRating, Count))
# Kết quả Join:        (MovieID, (Genre, (SumRating, Count)))
joined_genre_data = movie_genres_rdd.join(movie_rating_totals)

In [10]:
# 5. Gom nhóm theo Thể Loại (Genre)
# Chuyển đổi Key từ MovieID sang Genre để tính toán tổng thể cho từng thể loại
genre_stats = joined_genre_data.map(lambda x: (x[1][0], x[1][1])).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [11]:
# 6. Tính điểm trung bình cho từng thể loại
# Output: (Genre, (AverageRating, TotalRatings))
final_genre_metrics = genre_stats.mapValues(lambda x: (x[0] / x[1], x[1]))

In [12]:
# 7. Hiển thị kết quả
results = final_genre_metrics.collect()

In [13]:
for genre, (avg, total) in results:
    # Format: Genre - AverageRating (TotalRatings)
    print(f"{genre} - AverageRating: {avg:.2f} (TotalRatings: {total})")

Sci-Fi - AverageRating: 3.73 (TotalRatings: 54)
Horror - AverageRating: 4.00 (TotalRatings: 2)
Thriller - AverageRating: 3.70 (TotalRatings: 27)
Family - AverageRating: 3.67 (TotalRatings: 18)
Action - AverageRating: 3.71 (TotalRatings: 54)
Drama - AverageRating: 3.76 (TotalRatings: 128)
Adventure - AverageRating: 3.63 (TotalRatings: 83)
Mystery - AverageRating: 4.00 (TotalRatings: 2)
Crime - AverageRating: 3.81 (TotalRatings: 42)
Film-Noir - AverageRating: 4.36 (TotalRatings: 7)
Fantasy - AverageRating: 3.86 (TotalRatings: 29)
Biography - AverageRating: 3.56 (TotalRatings: 25)
